# 順運動学、逆運動学を用いた動作生成

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>順運動学、逆運動学を使ったHSRのアームの駆動方法を学習します</p>
</div> 

# Motion generation using forward and inverse kinematics

<div class="alert alert-block alert-info">
    <b>Objective</b>
    <p>In this notebook, we will learn how to control the arm of the HSR using forward and inverse kinematics.</p>
</div>

# 順運動学を用いた動作生成

「順運動学」とは、関節の変位からロボットの手先の位置・姿勢を求めることです。アームの各関節角度を指定してHSRを動かしてみましょう。

# Motion generation using forward kinematics

"Forward kinematics" is used to find the position and orientation of the robot's hand from the displacement of the joints. Let's move the arm of the HSR by specifying the joint angles.

必要なライブラリのインポートと、初期化を行います。

Import and initialize the required libraries:

In [ ]:
import math
import moveit_commander
import rospy
import tf
from utils import *
rospy.init_node("arm")

アームとして定義されている各関節の名称を配列として取得します。以降の各コマンドでは、この配列の順序に対応させて各関節値を設定します。

ここで`arm`はロボットのアームの関節角を管理する変数です。

Get the name of the joints defined as the arm group as an array. The subsequent commands set joint values according to the order of this array.

Here, `arm` is a variable that handles the robot arm:

In [ ]:
arm.get_active_joints()

各関節に対応するリンクがロボットのどの位置にあるか確認しましょう。

rvizを立ち上げて、「TF」を展開して「Frames」から興味のあるリンク名にチェックを入れてその位置を確認しましょう。

Let's check the names of the joints on the robot.

Launch RViz, expand "TF", check the link name you are interested in from "Frames" item, and check its location.

In [ ]:
%%script bash --bg
rviz -d data/4_arm.rviz > /dev/null 2>&1

![title](./imgs/4_model_and_tf.png)

各座標は赤緑青(RGB)の順番でx軸、y軸、z軸を表しています。

![title](./imgs/4_model_and_tf.png)

Each coordinate represents the x-axis, y-axis, and z-axis in the order of red, green, and blue.

各関節角の現在値を取得します。関節角度の単位はラジアンで返ってきます。

Let's get the current value of each joint angle. Joint angles are returned as radians:

In [ ]:
arm.get_current_joint_values()

試しに指定した関節値にアームを駆動してみます。

まずは、目標の関節値を設定します。

Try moving the arm to the joint value specified.

First, set the target joint values:

In [ ]:
arm.set_joint_value_target([0, 0, 0, 0, 0, 0])

目標を設定後「go()」コマンドを実行することで、アームが動きます。

After setting the target, execute the "go()" command to move the arm:

In [ ]:
arm.go()

例えば、以下のように設定すると昇降軸を動作させることができます。

For example, you can extend the lifting axis by sending the command as follows:

In [ ]:
arm.set_joint_value_target([0.5, 0, 0, 0, 0, 0])
arm.go()

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>「モノを拾う」アームの動きを作ってみましょう。</p>
</div> 

<div class="alert alert-block alert-info">
    <b>Task</b>
    <p>Create an arm motion to "pick up an object".</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。 Write your code here:


# 逆運動学を用いた動作生成

アームの各関節を設定して動きを作るのは案外大変です。「逆運動学」を使うと手先の目標座標から各関節に設定すべき角度を自動で算出することができます。

# Motion generation using inverse kinematics

You probably understand that it is unexpectedly difficult to set joint values of the arm to make a desired movement. By using "Inverse Kinematics", we can set the angle of the joints automatically by setting the target position of the hand.

「順運動学を用いた動作生成」では、HSRのアームのみを動かしていました。しかし、アームだけでは遠くの物を掴めないなどの問題があります。

HSRはモバイルマニピュレータと呼ばれ、台車の上にアームが搭載されています。以降は、台車とアームを用いた全身の制御を行います。

In addition, in "Motion generation using forward kinematics", only the HSR arm was used. However,  we cannot grab distant objects by moving the arm alone.

The HSR is a "mobile manipulator" that has the arm mounted on the moving base. We will show the combined control using the moving base and the arm from now on.

ロボットの手先(エンドエフェクタ)の名前を確認します。アームの先にあるハンドのリンクが設定されているはずです+。

ここで`whole_body`はロボットの頭を除く全関節角を管理する変数です。

Check the name of the robot's fingers (=end effectors). The hand link at the end of the arm should be set.

Here, `whole_body` is a variable that handles all joint angles of the robot except for the head.

In [ ]:
whole_body.get_end_effector_link()

## 各関節情報の確認

## Checking joint information

ロボットの各関節の角度は、現在、以下の値になっています。

The angles of the joints of the robot are currently set to the following values:

In [ ]:
whole_body.get_current_joint_values()

上記の関節角度は、以下の各関節名に対応します。

The above joint angles correspond to each of the following joint names:

In [ ]:
whole_body.get_joints()

ここで、ロボットの構造は既知なので、ロボットの構造に現在の各関節角度を当てはめ、座標変換を繰り返せばロボットの手先座標を計算できます。

Here, since the structure of the robot is known, the position of the robot's hand can be calculated by applying each current joint angle to the structure of the robot and repeating the coordinate transformations.

HSRの構造情報を表示させてみましょう。

リンクの親子関係と、その相対座標を表示した画像が得られます。四角形で表されているのがリンクで、円で表されているのが関節です。

大きな画像なので、スクロールして閲覧してください。

Let's display the HSR's structural information.

You can get an image showing the parent-child relationship of the link and its relative coordinates. The links are represented by squares, and the joints are represented by circles.

Since it is a large image, please scroll to view it:

In [ ]:
!rosparam get -p /robot_description > /tmp/robot.urdf
!urdf_to_graphiz /tmp/robot.urdf && dot -T png hsrb.gv -o /tmp/robot.png
from IPython.display import Image
with open('/tmp/robot.png','rb') as file:
    display(Image(data=file.read(), unconfined=True))

以下のコマンドを実行することで、順運動学によって計算されたエンドエフェクタ('hand_palm_link')の位置姿勢を知ることができます。

By executing the following command, you can calculate the position and orientation of the end effector ('hand_palm_link') calculated using forward kinematics:

In [ ]:
get_relative_coordinate("map", "hand_palm_link")

手先座標は、3次元の位置と四元数（quaternion）による姿勢で表現されています。四元数の詳細が気になる方は調べてみてください。

The hand coordinates are represented by a 3D position and a quaternion. Please search on the internet, if you are interested in details about "quaternions".

## 逆運動学の計算

## Inverse kinematics calculations

目標とする手先座標を与えられた時に、そこからアームの各関節角を逆算することを「逆運動学を解く」と言います。

まずは目標の手先座標を設定します。「map」という座標を基準にして、位置：(x,y,z)=(0.2,0.4,1.0)、姿勢：(roll, pitch, yaw)=(180, 90, 0)をエンドエフェクタの目標値としています。

ここで姿勢はx軸, y軸, z軸の順番でroll, pitch, yawの値で回転させ、その値を四元数に変換しています。

Given the target coordinates of the hand, calculating joint angles of the arm is called "solving inverse kinematics".

First, set the target coordinates. The target of the end effector is "position: (x, y, z)=(0.2,0.4,1.0)" and "orientation: (roll, pitch, yaw)=(180, 90, 0)" on the base coordinate "map".

Here, the posture is rotated by roll, pitch and yaw in the order of x-axis, y-axis and z-axis. The values are converted into quaternion:

In [ ]:
from geometry_msgs.msg import PoseStamped

p = PoseStamped()

# 基準座標を設定 Set the base coordinate
p.header.frame_id = "map"

# 位置を設定 Set target position
p.pose.position.x = 0.2
p.pose.position.y = 0.4
p.pose.position.z = 1.0

# ロール、ピッチ、ヨーの順番で回転し、クオータニオンに変換 Convert Euler to Quaternion
p.pose.orientation = quaternion_from_euler(180, 90, 0)

# 目標位置姿勢をセット Set position and orientation target
whole_body.set_pose_target(p)

目標を設定後、以下を実行すると、逆運動学の計算が始まります。

計算が成功すると算出された軌道が返ってきます。`plan`には、各時間ごとの全関節角の目標値の情報(時間軌道と呼びます)が詰まっています。

逆運動学の解が得られなかった場合（目標の手先座標の設定が適切でなかった場合）は、長さゼロの配列が返ってきますので、目標を修正して再計算してください。

After setting the target, we will start the inverse kinematics calculation.

If the calculation is successful, the calculated trajectory will be returned. Variable `plan` stores the information on the target values of all joint angles for each time (called "time trajectory").

If the inverse kinematics solution is not obtained (if the target hand coordinates are not set properly), an array of zero length will be returned. Please correct the target position and recalculate in such a case:

In [ ]:
plan = whole_body.plan()
plan

算出された時間軌道を実際に実行します。

生成された軌道を引数として与えて以下を実行することで、アームが逆運動学の計算結果に従って動きます。

We actually execute the calculated time trajectory.

The arm moves according to the calculation result of inverse kinematics by giving the generated trajectory as an argument to the  `execute()` function:

In [ ]:
whole_body.execute(plan)

「TF」を展開して、「Frames」の中の「map」と「hand_palm_link」を表示させてみましょう。

エンドエフェクタが目標の位置姿勢になっていることを確認しましょう。

![title](./imgs/4_map_ee_tf.png)

Expand "TF" to display "map" and "hand_palm_link" in "Frames".

Confirm that the end effector is in the target position and orientation.

![title](./imgs/4_map_ee_tf.png)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>エンドエフェクタの目標位置姿勢を変えて、ロボットを動かしてみましょう。</p>
</div>

<div class="alert alert-block alert-info">
    <b>Task</b>
    <p>Move the robot by changing the target position of the end effector.</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。 Write your code here


# レゴブロックを拾ってみる

# Pick up a Lego block

上記の処理をまとめたmove_wholebody_ikという関数を用いてみましょう。引数は3次元位置と手先の回転角度です。

```python
import moveit_commander
from geometry_msgs.msg import PoseStamped

# moveitでの制御対象として全身制御を指定
whole_body = moveit_commander.MoveGroupCommander("whole_body_light")
whole_body.allow_replanning(True)
whole_body.set_workspace([-3.0, -3.0, 3.0, 3.0])

def move_wholebody_ik(x, y, z, roll, pitch, yaw):
    u"""ロボットを全身の逆運動学で制御する関数

    引数：
        x (float): エンドエフェクタの目標x値 [m]
        y (float): エンドエフェクタの目標y値 [m]
        z (float): エンドエフェクタの目標z値 [m]
        roll (float): エンドエフェクタの目標roll値 [deg]
        pitch (float): エンドエフェクタの目標pitch値 [deg]
        yaw (float): エンドエフェクタの目標yaw値 [deg]

    返り値:
        正しく動作すればTrue, そうでなければFalse

    """

    p = PoseStamped()

    # "map"座標を基準座標に指定
    p.header.frame_id = "/map"

    # エンドエフェクタの目標位置姿勢のx,y,z座標をセットします
    p.pose.position.x = x
    p.pose.position.y = y
    p.pose.position.z = z

    # オイラー角をクオータニオンに変換します
    p.pose.orientation = quaternion_from_euler(roll, pitch, yaw)

    # 目標位置姿勢をセット
    whole_body.set_pose_target(p)
    return whole_body.go()

```

関数内でオイラー角から四元数に変換することで、人間が理解しやすいオイラー角によって手先の方向を設定できるようにしています。

例えば、上から物を掴みたい時は(roll, pitch, yaw) = (180,0,手先の回転角度)、横からは(180,90,手先の回転角度)というように設定すれば良いです。

Let's use a function called `move_wholebody_ik` that summarizes the above processing. The arguments are the 3D position and the rotation angle of the hand.

```python
import moveit_commander
from geometry_msgs.msg import PoseStamped

# Specify whole body control as the control target of moveit
whole_body = moveit_commander.MoveGroupCommander ("whole_body_light")
whole_body.allow_replanning(True)
whole_body.set_workspace ([-3.0, -3.0, 3.0, 3.0])

def move_wholebody_ik(x, y, z, roll, pitch, yaw):
    u"""Function that controls the robot by inverse kinematics of the whole body

    argument:
        x(float): End effector target x value [m]
        y(float): End effector target y value [m]
        z(float): End effector target z value [m]
        roll(float): End effector target roll value [deg]
        pitch(float): End effector target pitch value [deg]
        yaw(float): End effector target yaw value [deg]

    Return value:
        True if working correctly, False otherwise
    """

    p = PoseStamped ()

    # Specify "map" coordinates as base coordinate
    p.header.frame_id = "map"

    # Target position of end effector Set the x, y, z
    p.pose.position.x = x
    p.pose.position.y = y
    p.pose.position.z = z

    # Convert euler angle to a quarternion
    p.pose.orientation = quaternion_from_euler(roll, pitch, yaw)

    # Set target position and posture
    whole_body.set_pose_target(p)
    return whole_body.go ()
`` ```

In the function, we convert Euler angles to quaternions. This makes possible to set the direction of the hand by Euler angles that are easy for humans to understand.

For example, if you want to grab an object from orientation above, set (roll, pitch, yaw)=(180, 0, hand rotation angle).
And from the side (180, 90, hand rotation angle).

シミュレータ内に物体を出現させて、HSRに拾わせてみましょう。以下のコマンドでどのような物体があるか表示します。

Let's make an object appear in the simulator and make the HSR pick it up. The following command will display what kind of objects exist in the current scene:

In [ ]:
get_object_list()

今回は、レゴブロック「e_lego_duplo」をHSRの前方0.4mの位置に出現させます。

This time, we will make the Lego block "e_lego_duplo" appear 0.4m in front of the HSR:

In [ ]:
# レゴブロックを(x, y, z) = (0.4, 0.0, 0.0)の位置に出現させる
put_object("e_lego_duplo", 0.4, 0.0, 0.0)

ハンドを最大角度まで開きます。

Open the hand to the maximum angle:

In [ ]:
# 引数として1.0を与えると開き、0.0を与えると閉じる
move_hand(1.0)

逆運動学を使ってHSRの手先を前方0.4mの位置に動かします。

Use inverse kinematics to move the HSR's end effector to a position 0.4m forward:

In [ ]:
move_wholebody_ik(0.45, -0.05, 0.1, 180, 0, 0)

ハンドを閉じます。

Close the hand:

In [ ]:
move_hand(0.0)

アームを初期姿勢に戻します。

Return the arm to its initial position:

In [ ]:
move_arm_init()

レゴブロックを取るのに失敗した場合、以下のコマンドでレゴブロックを消して、再度出現させると元の取りやすい位置に戻ります。再挑戦してみましょう。

If you fail to pick up the Lego block, then delete the Lego block with the following command and make it reappear at the original position from where it is easy to pick it up from:

In [ ]:
delete_object("e_lego_duplo")

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>カンを机の上に場所に出現させて、同様にカンを拾うプログラムを書いてみましょう。</p>
    <p>下のコマンドでカンを出現させたり、消したりすることができます。</p>
</div>

<div class="alert alert-block alert-info">
    <b>Task</b>
    <p>Spawn a can on the desk and write code to pick up the can.</p>
    <p>Use the following commands to spawn and delete the can.</p>
</div>

In [ ]:
put_object("tomato_soup_can", 1.1, 1.65, 0.45)

In [ ]:
delete_object("tomato_soup_can")

In [ ]:
# 自分で考えてみましょう。この下に入力できます。 Write your code here:


<div class="alert alert-block alert-info">
    <b>次の学習</b>
    <p>この章では物体の位置を人間が教えてあげていました。</p>
    <p>次章ではカメラ画像から物体を認識し、自動で物体を拾うプログラムを書いてみましょう。</p>
</div>

<div class="alert alert-block alert-info">
    <b>What's next?</b>
    <p>In this notebook we taught the robot about the object positions.</p>
    <p>In the next notebook, we will learn how to detect an object using the camera image and write code to pick up the object automatically.</p>
</div>